In [82]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import FloatImage
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import time
import numpy as np

In [83]:
reactors = pd.read_csv('../data/Cleaned Data - Copy/all_reactors_refilled.csv')
reactors.head()

,Status,Country,ReactorCode,Reactor Name,Type,Grid Connection,Online,Commercial operation,Shutdown,Offline,Latitude,Longitude,Link1,Link2,Link3,Power from Nuclear
0,Operational,France,FR-17,ST. LAURENT B-1,PWR,1/1/1981,"January, 1981",8/1/1983,NaN,None (Currently operational),47.720000,1.577500,https://en.wikipedia.org/wiki/Saint-Laurent_Nu...,http://www.world-nuclear.org/reactor/default.a...,NaN,0.71
1,Operational,France,FR-23,ST. LAURENT B-2,PWR,6/1/1981,"June, 1981",8/1/1983,NaN,None (Currently operational),47.720000,1.577500,https://en.wikipedia.org/wiki/Saint-Laurent_Nu...,http://www.world-nuclear.org/reactor/default.a...,NaN,0.71
2,Shut Down,Italy,IT-3,ENRICO FERMI,PWR,10/1/1964,"October, 1964",1/1/1965,7/1/1990,"July, 1990",45.184094,8.277556,https://en.wikipedia.org/wiki/Enrico_Fermi,https://en.wikipedia.org/wiki/Enrico_Fermi_Nuc...,NaN,0.00
3,Shut Down,Japan,JP-5,FUKUSHIMA-DAIICHI-1,BWR,11/1/1970,"November, 1970",3/1/1971,5/1/2011,"May, 2011",37.423060,141.033060,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://www.world-nuclear.org/information-libr...,0.03
4,Shut Down,Japan,JP-9,FUKUSHIMA-DAIICHI-2,BWR,12/1/1973,"December, 1973",7/1/1974,5/1/2011,"May, 2011",37.423060,141.033060,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://www.world-nuclear.org/information-libr...,0.03


In [84]:
conditions = [
    reactors['Status'] == 'Shut Down',
    reactors['Status'] == 'Operational'
    ]

values = ['lightgray', 'darkgreen']

reactors['color'] = np.select(conditions, values)

In [85]:
geometry = [Point(xy) for xy in zip(reactors.Longitude, reactors.Latitude)]
reactor_geo = gpd.GeoDataFrame(reactors, crs="EPSG:3857", geometry=geometry)

In [86]:
reactor_map = folium.Map(location = [0, 0], zoom_start = 1)
folium.TileLayer('cartodbpositron').add_to(reactor_map)

In [87]:
marker_cluster = MarkerCluster().add_to(reactor_map)

for row_index, row_values in reactor_geo.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = row_values['Reactor Name']
    marker = folium.Marker(
        location = loc,
        icon = folium.Icon(color=row_values['color'],icon="bolt", prefix='fa'),
#        radius=100,
        popup = pop,
        fill=True,
        color=row_values['color']) 
    
    marker.add_to(marker_cluster)
    
#    marker.add_to(ntest_map)
    
reactor_map

In [88]:
reactor_map2 = folium.Map(location = [0, 0], zoom_start = 1)
folium.TileLayer('cartodbpositron').add_to(reactor_map2)

In [89]:
marker_cluster = MarkerCluster().add_to(reactor_map2)

for row_index, row_values in reactor_geo.iterrows():
    name = row_values['Reactor Name']
    state = row_values['Country']
    date_online = 'Connected to grid: ' + row_values['Online']
    date_offline = 'Date of shutdown: ' + row_values['Offline']
    link1 = row_values['Link1']
    link2 = row_values['Link2']
    hue = row_values['color']
    html="""
        <h2>{reactor}</h2>
        <h3>{country}</h3><br>
        {on}<br>
        {off}<br>
        <p>
        Further reading:<br>
        <a href="{external1}">Link 1</a><br>
        <a href="{external2}">Link 2</a><br>
        </p>
        """
    html = html.format(reactor=name, country=state, external1 = link1, 
                       on=date_online, off=date_offline, external2 = link2)
    iframe = folium.IFrame(html=html, width=300, height=300)
    pop = folium.Popup(iframe, max_width=2650)
    loc = [row_values['Latitude'], row_values['Longitude']]
    marker = folium.Marker(
        location = loc,
        icon = folium.Icon(color=hue,icon="bolt", prefix='fa'),
#        radius=100,
        popup = pop,
        fill=True,
        color=hue)
    
    marker.add_to(marker_cluster)
    
#    marker.add_to(ntest_map)
    
reactor_map2

In [90]:
reactor_geo

,Status,Country,ReactorCode,Reactor Name,Type,Grid Connection,Online,Commercial operation,Shutdown,Offline,Latitude,Longitude,Link1,Link2,Link3,Power from Nuclear,color,geometry
0,Operational,France,FR-17,ST. LAURENT B-1,PWR,1/1/1981,"January, 1981",8/1/1983,NaN,None (Currently operational),47.720000,1.577500,https://en.wikipedia.org/wiki/Saint-Laurent_Nu...,http://www.world-nuclear.org/reactor/default.a...,NaN,0.71,darkgreen,POINT (1.57750 47.72000)
1,Operational,France,FR-23,ST. LAURENT B-2,PWR,6/1/1981,"June, 1981",8/1/1983,NaN,None (Currently operational),47.720000,1.577500,https://en.wikipedia.org/wiki/Saint-Laurent_Nu...,http://www.world-nuclear.org/reactor/default.a...,NaN,0.71,darkgreen,POINT (1.57750 47.72000)
2,Shut Down,Italy,IT-3,ENRICO FERMI,PWR,10/1/1964,"October, 1964",1/1/1965,7/1/1990,"July, 1990",45.184094,8.277556,https://en.wikipedia.org/wiki/Enrico_Fermi,https://en.wikipedia.org/wiki/Enrico_Fermi_Nuc...,NaN,0.00,lightgray,POINT (8.27756 45.18409)
3,Shut Down,Japan,JP-5,FUKUSHIMA-DAIICHI-1,BWR,11/1/1970,"November, 1970",3/1/1971,5/1/2011,"May, 2011",37.423060,141.033060,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://www.world-nuclear.org/information-libr...,0.03,lightgray,POINT (141.03306 37.42306)
4,Shut Down,Japan,JP-9,FUKUSHIMA-DAIICHI-2,BWR,12/1/1973,"December, 1973",7/1/1974,5/1/2011,"May, 2011",37.423060,141.033060,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://en.wikipedia.org/wiki/Fukushima_Daiich...,https://www.world-nuclear.org/information-libr...,0.03,lightgray,POINT (141.03306 37.42306)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,Shut Down,United States,US-001,SHIPPINGPORT,PWR,12/1/1957,"December, 1957",5/1/1958,10/1/1982,"October, 1982",40.621110,-80.435280,https://en.wikipedia.org/wiki/Shippingport_Ato...,https://www.world-nuclear.org/information-libr...,NaN,0.19,lightgray,POINT (-80.43528 40.62111)
619,Shut Down,United States,US-322,SHOREHAM,BWR,8/1/1986,"August, 1986",8/1/1986,5/1/1989,"May, 1989",40.961110,-72.865000,https://en.wikipedia.org/wiki/Shoreham_Nuclear...,https://www.world-nuclear.org/information-libr...,NaN,0.19,lightgray,POINT (-72.86500 40.96111)
620,Shut Down,United States,US-344,TROJAN,PWR,12/1/1975,"December, 1975",5/1/1976,11/1/1992,"November, 1992",46.038330,-122.885000,https://en.wikipedia.org/wiki/Trojan_Nuclear_P...,https://www.world-nuclear.org/information-libr...,NaN,0.19,lightgray,POINT (-122.88500 46.03833)
621,Shut Down,United States,US-271,VERMONT YANKEE,BWR,9/1/1972,"September, 1972",11/1/1972,12/1/2014,"December, 2014",42.778890,-72.513060,https://en.wikipedia.org/wiki/Vermont_Yankee_N...,https://www.world-nuclear.org/information-libr...,NaN,0.19,lightgray,POINT (-72.51306 42.77889)


In [91]:
reactors['color'].value_counts()

darkgreen    439
lightgray    184
Name: color, dtype: int64